<a href="https://colab.research.google.com/github/GhostScientist/Colab/blob/main/MCPExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Context Protocol (MCP) Demonstration

This notebook demonstrates the core concepts of the **Model Context Protocol (MCP)**, an emerging standard for AI model interactions that promises to do for AI what REST did for web services.

## What is MCP?

MCP creates a standardized interface between applications and AI models, abstracting away provider-specific implementations. This allows developers to:

- Write code that works with multiple AI providers without modification
- Switch between models based on cost, capabilities, or performance
- Transfer conversation context seamlessly between different models
- Future-proof applications against changes in provider APIs

Think of MCP as a "universal adapter" for AI models - write once, run everywhere. Need a new model? Build the model-specific plug, then play!

In [ ]:
# MCP Demonstration: Standardizing AI Model Interactions
# ----------------------------------------------------------

# Install required packages
!pip install openai anthropic cohere -q

# Import necessary libraries
import os
import json
from typing import List, Dict, Any, Optional

from openai import OpenAI

from anthropic import Anthropic

## The Problem MCP Solves

Before diving into our MCP implementation, let's consider the current state of AI integration:

Each AI provider has their own unique API structure:
- Different parameter names (`messages` vs `prompt`)
- Different authentication methods
- Different response formats
- Different context handling mechanisms
- Different nomenclature for roles (`user` vs `human`)

This fragmentation creates vendor lock-in, increases development time, and makes it difficult to benchmark or switch providers. The cells below demonstrate a solution to this problem through MCP.

In [ ]:
# Add your API keys securely using environment variables
# These will only exist for this Colab session - nothing is stored. If you want further control, feel free to make a copy of this notebook for yourself.

# OpenAI API key
openai_api_key = input("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Anthropic API key (optional)
anthropic_api_key = input("Enter your Anthropic API key (or press Enter to skip): ")
if anthropic_api_key:
    os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key

Enter your OpenAI API key: 
Enter your Anthropic API key (or press Enter to skip): 


## MCP Core Concepts

The foundation of for the protocol is a standardized interface that abstracts away provider-specific details. Our example implementation focuses on three key principles:

1. **Unified Message Format**: A standard structure for messages regardless of provider
2. **Context Management**: Consistent handling of conversation history
3. **Provider Abstraction**: A common interface that works across multiple AI models

Our `MCPClient` base class defines this interface, which provider-specific implementations will adapt to their respective APIs.

In [ ]:
class MCPClient:
    def __init__(self, provider_name: str = "unknown"):
        self.provider_name = provider_name
        self.context = []

    def add_message(self, role: str, content: str) -> None:
        standard_role = role.lower()
        if standard_role not in ["system", "user", "assistant"]:
            raise ValueError(f"Invalid role: {role}. Must be 'system', 'user', or 'assistant'")

        self.context.append({"role": standard_role, "content": content})

    def generate_response(self, prompt: Optional[str] = None,
                          system: Optional[str] = None) -> str:
        """
        Generate a response using the current context

        Args:
            prompt: Optional new user message to add before generating a response
            system: Optional system message to set or update

        Returns:
            The AI model's response text
        """
        raise NotImplementedError("Subclasses must implement this method")

    def get_context(self) -> List[Dict[str, str]]:
        return self.context.copy()

    def set_context(self, context: List[Dict[str, str]]) -> None:
        self.context = context.copy()

    def clear_context(self) -> None:
        self.context = []

## Provider-Specific Adapters

With our standard interface defined, we now need to create "adapters" (those model-specific plugs I mentioned before!) for each AI provider. These adapters:

- Translate between our standard MCP format and provider-specific APIs
- Handle provider-specific requirements (like different parameter names)
- Manage differences in context handling between providers

This abstraction of integration is the value proposition of MCP. This pattern is similar to how database ORMs abstract away differences between database engines, allowing your application code to remain unchanged regardless of the underlying database.

## Model-Specific Considerations

Even within the same provider, different models can have different requirements. For example, OpenAI's older models use a different parameter structure than their newer ones, and models like o1-mini don't support system messages.

MCP should handle these differences transparently to the application. This is similar to how REST handles different API versions or capabilities without requiring client applications to change.

The following example demonstrates how our MCP implementation handles model-specific differences:

In [ ]:
class OpenAIMCPClient(MCPClient):

    def __init__(self, api_key: Optional[str] = None, model: str = "gpt-4o-mini"):
        super().__init__(provider_name="openai")
        self.model = model

        # Use provided API key or get from environment variable
        if api_key:
            self.client = OpenAI(api_key=api_key)
        else:
            self.client = OpenAI()

    def generate_response(self, prompt: Optional[str] = None,
                          system: Optional[str] = None) -> str:
        # Example of sppecial handling for models that don't support system role
        models_without_system_role = ["o1-mini"]
        supports_system = self.model not in models_without_system_role

        if system and supports_system:
            has_system = any(msg["role"] == "system" for msg in self.context)

            if has_system:
                for i, msg in enumerate(self.context):
                    if msg["role"] == "system":
                        self.context[i] = {"role": "system", "content": system}
                        break
            else:
                self.context.insert(0, {"role": "system", "content": system})
        elif system and not supports_system:
            prompt = f"System instruction: {system}\n\nUser query: {prompt}"

        if prompt:
            self.add_message("user", prompt)

        api_messages = self.context.copy()
        if not supports_system:
            api_messages = [msg for msg in api_messages if msg["role"] != "system"]

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=api_messages
            )

            assistant_message = response.choices[0].message.content
            self.add_message("assistant", assistant_message)

            return assistant_message

        except Exception as e:
            return f"Error: {str(e)}"

In [ ]:
openai_client = OpenAIMCPClient(model="gpt-4o-mini")

response = openai_client.generate_response(
    prompt="What is the Model Context Protocol?",
    system="You are a helpful AI assistant that explains technical concepts clearly and concisely."
)

print(f"Response from {openai_client.provider_name} ({openai_client.model}):\n")
print(response)

Response from openai (gpt-4o-mini):

The Model Context Protocol (MCP) is a framework used primarily in artificial intelligence and machine learning to enhance the interactions between models and their environments. It provides a structured way for models to understand and adapt to the context in which they operate, allowing for more effective decision-making and action.

Key components of the Model Context Protocol typically include:

1. **Context Representation**: This refers to the information about the environment, constraints, goals, and other relevant factors that the model needs to consider. This could include data about user preferences, situational variables, or external conditions.

2. **Adaptation Mechanism**: MCP includes strategies for models to adjust their behavior based on the context they are given. This might involve modifying parameters, switching between different algorithms, or changing the way information is processed.

3. **Feedback Loop**: The protocol often inco

In [ ]:
class AnthropicMCPClient(MCPClient):
    """Anthropic-specific implementation of the MCP client"""

    def __init__(self, api_key: Optional[str] = None, model: str = "claude-3-5-sonnet-20241022"):
        super().__init__(provider_name="anthropic")
        self.model = model

        # Use provided API key or get from environment variable
        if api_key:
            self.client = Anthropic(api_key=api_key)
        else:
            self.client = Anthropic()

    def generate_response(self, prompt: Optional[str] = None,
                          system: Optional[str] = None) -> str:
        """Generate a response using Anthropic's API"""

        if prompt:
            self.add_message("user", prompt)

        anthropic_messages = []

        for msg in self.context:
            if msg["role"] != "system":
                role = "user" if msg["role"] == "user" else "assistant"
                anthropic_messages.append({"role": role, "content": msg["content"]})


        if not anthropic_messages:
            anthropic_messages = [{"role": "user", "content": "Hello"}]

        try:
            response = self.client.messages.create(
                model=self.model,
                max_tokens=1024,
                messages=anthropic_messages
            )

            has_system = any(msg["role"] == "system" for msg in self.context)
            if has_system or system:
                print("Note: System message was ignored for Anthropic API call")

            assistant_message = response.content[0].text
            self.add_message("assistant", assistant_message)

            return assistant_message

        except Exception as e:
            return f"Error: {str(e)}"

In [ ]:
def test_anthropic_direct():
    from anthropic import Anthropic

    client = Anthropic()

    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1024,
            messages=[
                {"role": "user", "content": "Hello, please respond with one word: TESTING"}
            ]
        )
        print("API call successful!")
        print(f"Response: {response.content[0].text}")
        return True
    except Exception as e:
        print(f"API call failed: {str(e)}")
        return False

test_result = test_anthropic_direct()

API call successful!
Response: TESTING


## The Power of Provider Switching

One of the major benefits of MCP is the ability to switch providers with minimal code changes. This enables:

- A/B testing between different models
- Fallback strategies when one provider is unavailable
- Cost optimization by routing different tasks to different providers
- Taking advantage of each provider's strengths

The following sentiment analysis example demonstrates how the same code can work with different providers through our MCP abstraction:

In [ ]:
def analyze_sentiment(text: str, client: MCPClient) -> str:
    """
    Analyze the sentiment of text using any MCP-compatible client

    Args:
        text: The text to analyze
        client: Any MCP-compatible client

    Returns:
        The sentiment analysis result
    """

    client.clear_context()


    system_prompt = """
    You are a sentiment analysis assistant. Analyze the sentiment of the provided text
    and respond with exactly one word: POSITIVE, NEGATIVE, or NEUTRAL.
    Provide no other text in your response.
    """


    result = client.generate_response(
        prompt=f"Analyze the sentiment of this text: {text}",
        system=system_prompt
    )

    return result

test_texts = [
    "I absolutely love this product! It's the best purchase I've made all year.",
    "The service was terrible and the staff was rude. I'm never going back.",
    "The movie was okay. It had some good moments but also some boring parts."
]

# Uncomment either OpenAI or Anthropic to test.

# Test with OpenAI
#print("SENTIMENT ANALYSIS WITH OPENAI:\n")
#for text in test_texts:
#    sentiment = analyze_sentiment(text, openai_client)
#    print(f"Text: {text}")
#    print(f"Sentiment: {sentiment}\n")


anthropic_client = AnthropicMCPClient()
print("\nSENTIMENT ANALYSIS WITH ANTHROPIC:\n")
for text in test_texts:
     sentiment = analyze_sentiment(text, anthropic_client)
     print(f"Text: {text}")
     print(f"Sentiment: {sentiment}\n")


SENTIMENT ANALYSIS WITH ANTHROPIC:

Note: System message was ignored for Anthropic API call
Text: I absolutely love this product! It's the best purchase I've made all year.
Sentiment: This text has a very positive sentiment. The use of enthusiastic language ("absolutely love"), superlatives ("best"), and exclamation points indicates strong positive emotions. The speaker expresses complete satisfaction with their purchase.

Note: System message was ignored for Anthropic API call
Text: The service was terrible and the staff was rude. I'm never going back.
Sentiment: This text has a strongly negative sentiment. The use of words like "terrible" and "rude" express clear dissatisfaction, and the statement "never going back" reinforces the negative experience and shows complete rejection of the establishment.

Note: System message was ignored for Anthropic API call
Text: The movie was okay. It had some good moments but also some boring parts.
Sentiment: This text expresses a mixed or neutral

## Context Portability: A Game Changer

Perhaps the most powerful feature of MCP is context portability - the ability to transfer conversation history between different providers. This enables:

- Seamless handoffs between models with different specialties
- Reducing costs by using more expensive models only when needed
- Resilience against provider outages
- Comparative analysis of model responses to the same context

The following example demonstrates context transfer between providers:

In [ ]:
# FYI: This example will only work if you have both API keys set up.

def demonstrate_context_transfer():

    openai_client = OpenAIMCPClient()
    openai_client.clear_context()

    print("Starting conversation with OpenAI:\n")

    openai_client.generate_response(
        system="You are a helpful assistant that provides concise information about planets.",
        prompt="Tell me about Mars."
    )

    print(f"[OpenAI]: {openai_client.context[-1]['content']}\n")

    openai_client.generate_response(prompt="What about its moons?")
    print(f"[OpenAI]: {openai_client.context[-1]['content']}\n")

    conversation_context = openai_client.get_context()

    try:
        anthropic_client = AnthropicMCPClient()

        anthropic_client.set_context(conversation_context)

        print("Continuing conversation with Anthropic:\n")
        response = anthropic_client.generate_response(
            prompt="How does gravity on Mars compare to Earth?"
        )

        print(f"[Anthropic]: {response}")

    except Exception as e:
        print(f"Couldn't test with Anthropic: {str(e)}")
        print("You may need to set up your Anthropic API key to run this example.")


demonstrate_context_transfer()

Starting conversation with OpenAI:

[OpenAI]: Mars is the fourth planet from the Sun in our solar system and is often called the "Red Planet" due to its reddish appearance, which is a result of iron oxide (rust) on its surface. Here are some key facts about Mars:

1. **Size and Structure**: Mars has a diameter of about 6,779 kilometers (4,212 miles), making it about half the size of Earth. It has a thin atmosphere, composed mostly of carbon dioxide.

2. **Moons**: Mars has two small moons, Phobos and Deimos, which are thought to be captured asteroids.

3. **Surface Features**: The planet features the largest volcano in the solar system, Olympus Mons, and a massive canyon system, Valles Marineris. It also has polar ice caps made of water and carbon dioxide.

4. **Water**: Mars has evidence of past water flows and currently has water ice beneath its surface, with seasonal dark streaks suggesting possible briny liquid water flows.

5. **Exploration**: Mars has been explored by numerous sp

## From Theory to Practice: MCP in Production

This notebook has demonstrated a simplified implementation of MCP concepts. In a production environment, you might:

1. Use a full-featured MCP library rather than building your own
2. Add advanced features like:
   - Streaming responses
   - Tool use standardization
   - Error handling normalization
   - Prompt template management

The key takeaway is that standardization through protocols like MCP creates significant advantages:

- **Reduced development time**: Integrate once, use many providers
- **Future-proofing**: Switch to better models as they emerge
- **Negotiating power**: Avoid vendor lock-in
- **Architectural flexibility**: Design multi-model systems easily

As AI becomes increasingly central to applications, the need for standardization becomes more critical. MCP represents an important step toward a more mature, interoperable AI ecosystem.